In [ ]:
# установка зависимостей (при первом запуске - обязательно)

%pip install ultralytics
%pip install Pillow

# 1 - Детектинг Рукописных сегментов

Тестирование происходит на примере одного изображения, необходимо переопределить путь к тестовому изображению вручную

In [2]:
# Имя директории
urimagedirectory=".../test_images"

# Путь к файлу
#img_path = f"ur_img_dir{filename}"

In [3]:
# Имя файла
from pathlib import Path
arrIm=[]
for image_file in sorted(Path(urimagedirectory).resolve().glob("*.png")):
    arrIm.append(image_file)  
for image_file in sorted(Path(urimagedirectory).resolve().glob("*.jpg")):
    arrIm.append(image_file)  

In [4]:
from ultralytics import YOLO

PATH = 'models/best-unprocessed.pt'
model = YOLO(PATH)
resArr=[]
for image_path in arrIm:
    res = model.predict(source=image_path, project='.', name='detected', exist_ok=True, save=True, show=False, show_labels=True, show_conf=False, conf=0.3)
    resArr.append(res)


image 1/1 C:\Users\vazon\Desktop\T1 Hackathon\git\test_images\4_t_5.png: 480x640 53 Unsigneds, 208.5ms
Speed: 16.2ms preprocess, 208.5ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
Results saved to detected

image 1/1 C:\Users\vazon\Desktop\T1 Hackathon\git\test_images\photo_2024-10-27_00-15-38.jpg: 480x640 20 Unsigneds, 1 Signed, 118.6ms
Speed: 0.0ms preprocess, 118.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to detected


## 2 - Кроппинг рукописных сегментов

In [5]:
from PIL import *
def extract_rectangles(image, rectangles):
    extracted_images = []
    
    for i in rectangles:
        x_ = i[0]
        y_ = i[1]
        w = i[2]
        h = i[3]
        x1 = x_ - w// 2
        y1 = y_ - h// 2
        x2 = x_ + w // 2
        y2 = y_ + h // 2
            
        cropped_img = image.crop((x1, y1, x2, y2))
        extracted_images.append(cropped_img)

    return extracted_images
rectImages=[]
signImages=[]

# Обработка изображений и извлечение объектов
for index, img_path in enumerate(arrIm):
    image = Image.open(img_path)  # Открытие изображения
    array_of_boxes = []
    sign = False  # Сброс флага на каждом изображении

    # Получение результатов для текущего изображения
    results = resArr[index]  
    if results:  # Проверка, что результаты не пустые
        det = results[0]  # Первый элемент списка с результатами
    
        for obj in det.boxes:  # Используем det.boxes для доступа к объектам
            obj_id = int(obj.cls[0])  # Получение ID объекта
            if obj_id == 1:
                sign = True  # Установка флага True, если объект с ID = 1 найден

            for b in obj.xywh:  # Извлечение боксов
                if obj_id == 0:  # Сохранение только боксов с ID = 0
                    array_of_boxes.append(b.tolist())

    signImages.append(sign)  # Сохранение True/False в signImages
    ans = extract_rectangles(image, array_of_boxes)  # Извлечение сегментированных изображений
    rectImages.append(ans)  # Добавление в rectImages
c=0
print(rectImages)
for i in rectImages:
    for j in i:
        c+=1
print(signImages)
print(c)



[[<PIL.Image.Image image mode=RGBA size=28x18 at 0x1940A46EFD0>, <PIL.Image.Image image mode=RGBA size=36x22 at 0x1940A24C050>, <PIL.Image.Image image mode=RGBA size=156x38 at 0x19478402B10>, <PIL.Image.Image image mode=RGBA size=28x18 at 0x194783FCD90>, <PIL.Image.Image image mode=RGBA size=54x20 at 0x194783FED50>, <PIL.Image.Image image mode=RGBA size=174x38 at 0x194780541D0>, <PIL.Image.Image image mode=RGBA size=130x34 at 0x19478057210>, <PIL.Image.Image image mode=RGBA size=168x48 at 0x194783E3F90>, <PIL.Image.Image image mode=RGBA size=28x14 at 0x19478058890>, <PIL.Image.Image image mode=RGBA size=116x26 at 0x194780572D0>, <PIL.Image.Image image mode=RGBA size=32x18 at 0x19478058650>, <PIL.Image.Image image mode=RGBA size=62x24 at 0x1940A083990>, <PIL.Image.Image image mode=RGBA size=136x40 at 0x1940A4804D0>, <PIL.Image.Image image mode=RGBA size=126x40 at 0x1940A480590>, <PIL.Image.Image image mode=RGBA size=60x24 at 0x1940A480650>, <PIL.Image.Image image mode=RGBA size=130x40 a